In [3]:
!pip install scikit-learn==1.0.2

/opt/anaconda3/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
     |████████████████████████████████| 7.8 MB 4.6 MB/s eta 0:00:01
ERROR: karateclub 1.0.24 has requirement gensim==3.8.3, but you'll have gensim 3.6.0 which is incompatible.
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.1
    Uninstalling scikit-learn-0.22.1:
      Successfully uninstalled scikit-learn-0.22.1


In [23]:
!pip install pyarrow

/opt/anaconda3/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
     |████████████████████████████████| 22.3 MB 1.2 MB/s eta 0:00:01


In [4]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [5]:
import pickle
import pandas as pd

In [6]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [13]:
year = 2021
month = 2

In [14]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [15]:
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [16]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [10]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [11]:
y_pred.mean()

16.191691679979066

In [17]:
df['prediction'] = y_pred

In [18]:
df.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number',
       'duration', 'ride_id', 'prediction'],
      dtype='object')

In [19]:
df_result = df[['ride_id', 'prediction']]

In [24]:
df_result.to_parquet(
    f"predictions_{year:04d}-{month:02d}.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)